In [1]:
import os, sys
import geopandas as gpd
import pandas as pd
# import rasterio as rio
# import numpy as np
# from shapely.geometry import Point
# import skimage.graph as graph
from geopy.geocoders import GeoNames, Nominatim

In [2]:
from shapely.geometry import Point

In [3]:
# sys.path.append('/home/wb514197/Repos/gostrocks/src') # gostrocks is used for some basic raster operations (clip and standardize)
# sys.path.append('/home/wb514197/Repos/GOSTNets_Raster/src') # gostnets_raster has functions to work with friction surface
# sys.path.append('/home/wb514197/Repos/GOSTnets') # it also depends on gostnets for some reason
# sys.path.append('/home/wb514197/Repos/INFRA_SAP') # only used to save some raster results
# sys.path.append('/home/wb514197/Repos/HospitalAccessibility/src') # only used to save some raster results

In [4]:
# import GOSTRocks.rasterMisc as rMisc
# import GOSTNetsRaster.market_access as ma
# from infrasap import aggregator

In [5]:
iso3 = 'LBR'

In [6]:
input_dir = "/home/public/Data/PROJECTS/Health" #
out_folder = os.path.join(input_dir, "output", iso3)
# out_folder = r"C:\repos\health-equity-access\notebooks\output"
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

### Destinations
Do some reformatting

In [7]:
lbr_master = pd.read_excel(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia.xlsx"))

In [8]:
# lbr_master = pd.read_excel(r"C:\repos\GOST_PublicGoods\Implementations\FY22\ACC_HHNGF_Health_Planning\input\Liberia.xlsx")

In [9]:
lbr_master[:3]

,County,District,health district,Location,New HF ID,UUID Validated (DHIS2),HF_name validated (DHIS2),UUID,HF Name,Population (2011),...,Rapid syphillis testing,HIV rapid testing,Urine (rapid) testing,Urine (protein) testing,Urine (glucose) testing,urine (ketone) testing,Dry blood spot collection for HIV viral load oe EID,Unnamed: 122,Unnamed: 123,Unnamed: 124
0,Bomi,Senjeh,Senjeh,Tubmanburg,LRY6M0,lF9FiiVCXMN,Ahmiddiya Clinic,lF9FiiVCXMN,Ahmiddiya Clinic,3459.000000,...,Yes (onsite),No,Yes (onsite),Yes (onsite),Yes (onsite),Yes (onsite),No,NaN,NaN,NaN
1,Bomi,Senjeh,Senjeh,Beafine town,LRKDF8,iBaC7MdHTa0,Beafine Community Clinic,iBaC7MdHTa0,Beafine Community Clinic,1791.375144,...,No,Yes (onsite),Yes (onsite),No,No,No,No,NaN,NaN,NaN
2,Bomi,Dowein,Dowein,Beh Town,LR4GA0,ZxwlZv1rg7E,Beh Town Clinic,ZxwlZv1rg7E,Beh Town Clinic,3699.903239,...,No,Yes (onsite),Yes (onsite),Yes (onsite),Yes (onsite),Yes (onsite),No,NaN,NaN,NaN


In [10]:
lbr_master['status'] = lbr_master['Status'].apply(lambda x: x.lower())

In [11]:
lbr_master['hf_type'] = lbr_master['HF Type'].apply(lambda x: x.lower())

In [12]:
lbr = lbr_master.loc[lbr_master.Status!="non-functional"].copy()

In [13]:
lbr_missing = lbr.loc[lbr.Lat.isna()].copy()

In [14]:
lbr_non_missing = lbr.loc[~(lbr.Lat.isna())].copy()

In [15]:
list(lbr_missing.columns)

['County',
 'District',
 'health district',
 'Location',
 'New HF ID',
 'UUID Validated (DHIS2)',
 'HF_name validated (DHIS2)',
 'UUID',
 'HF Name',
 'Population (2011)',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'F_Type by EPHS classification',
 'HF Type',
 'Owership',
 'Status',
 'Established by',
 'Locality Type',
 'Lat',
 'Long',
 'Q008_Manag_Authority',
 'facility_contact_person',
 'facility_contact_position',
 'facility_person_contact',
 'number available?',
 'facility_person_contact2',
 '# of inpatient beds',
 '# of dedicated maternity inpatient beds',
 'Vehicle or Functional Ambulance',
 'Access to ambulance',
 'Water source',
 'On premise water available',
 'Function Refrigerator for Vaccine storage',
 'Functional Incinerator',
 'Functional Pit',
 'Means of waste disposal',
 'Functional Computer',
 'Functional Triage',
 'Dedicated Isolation room',
 'Functional landline for communication',
 'Cellular Network',
 'Access to phone',
 'Functional Short Wave Radio',
 'Access 

In [16]:
lbr_missing.Location = lbr_missing.Location.astype(str)

In [17]:
lbr_non_missing[:2]

,County,District,health district,Location,New HF ID,UUID Validated (DHIS2),HF_name validated (DHIS2),UUID,HF Name,Population (2011),...,Urine (rapid) testing,Urine (protein) testing,Urine (glucose) testing,urine (ketone) testing,Dry blood spot collection for HIV viral load oe EID,Unnamed: 122,Unnamed: 123,Unnamed: 124,status,hf_type
0,Bomi,Senjeh,Senjeh,Tubmanburg,LRY6M0,lF9FiiVCXMN,Ahmiddiya Clinic,lF9FiiVCXMN,Ahmiddiya Clinic,3459.000000,...,Yes (onsite),Yes (onsite),Yes (onsite),Yes (onsite),No,NaN,NaN,NaN,functional,clinic
1,Bomi,Senjeh,Senjeh,Beafine town,LRKDF8,iBaC7MdHTa0,Beafine Community Clinic,iBaC7MdHTa0,Beafine Community Clinic,1791.375144,...,Yes (onsite),No,No,No,No,NaN,NaN,NaN,functional,clinic


### 2 Geocode

In [19]:
# new function
def geocode_missing_locations(missing_locations, location_col, admin_col, health_district_col , adm_gpd, admin_col2, country_code):
    """
    Uses GeoPY and Nominatim to locate the coords of missing locations. Prioritizes and runs a query that includes the location name and admin name. 
    It will also run and query that includes only the location if the first query is not successful. 

    :param missing_locations: Dataframe containing the missing locations
    :param location_col: The location name (string) column in the missing_locations DataFrame
    :param admin_col: The admin name (string) column in the missing_locations DataFrame
    :health_district_col: The health district (string) column in the missing_locations DataFrame
    :param adm_gpd: Dataframe containing the admin polygons
    :param admin_col2: The admin name (string) column in the adm_gpd DataFrame
    :param country_code: 2 letter ISO 3166-1 country code (string) to pass into query
    :returns: A copy of the missing_locations DataFrame with added columns
    """
    
    # copied from docs
    # import pandas as pd
    # df = pd.DataFrame({'name': ['paris', 'berlin', 'london']})

    # from geopy.geocoders import Nominatim
    # geolocator = Nominatim(user_agent="specify_your_app_name_here")

    # from geopy.extra.rate_limiter import RateLimiter
    # geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    # df['location'] = df['name'].apply(geocode)

    # df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
    geolocator = Nominatim(user_agent="health-equity")
    
    missing_locations_copy = missing_locations.copy()
    
    county_array = lbr_missing['County'].values

    for idx, row in missing_locations_copy.iterrows():
        #print(row['County'])
        if row[admin_col] not in county_array:
            print(f"warning, {row[admin_col]} is not a part of admin polygon dataset")
            raise Exception(f"warning, {row[admin_col]} is not a part of admin polygon dataset")
    
    i = 1
    for idx, row in missing_locations_copy.iterrows():
        print(f"Geocoding {idx}, {i} of {len(lbr_missing)}")
        # Check if there ia another entry with location
        lbr_copy = lbr_non_missing.loc[lbr_non_missing.Location==row.Location,['Lat', 'Long']]
        if len(lbr_copy)>0:
            res = lbr_copy.iloc[0]
            missing_locations_copy.loc[idx, "Long"] = res.Long
            missing_locations_copy.loc[idx, "Lat"] = res.Lat
            print("\tSuccess for Another facility with same Location")
            missing_locations_copy.loc[idx, "geocoding_method"] = "Another facility with same Location"
        else:
            if "Community" in row['Location']:
                lbr_missing.loc[idx, "Location"] = row[location_col].replace(" Community", "")
            try:
                #res = geolocator.geocode(row[location_col], country_codes=country_code)
                res = geolocator.geocode(row['Location'] + ', ' + row[admin_col], country_codes=country_code)
                # if within county
                if counties_gpd.loc[counties_gpd[admin_col2] == row[admin_col]].contains(Point(res.longitude,res.latitude)).values[0] == True:
                    missing_locations_copy.loc[idx, "Long"] = res.longitude
                    missing_locations_copy.loc[idx, "Lat"] = res.latitude
                    print("\tSuccess for Location name and county query")
                    missing_locations_copy.loc[idx, "geocoding_method"] = "Location name and county query"
                else:
                    #res = geolocator.geocode(row['Location'] + ', ' + row[admin_col], country_codes=country_code)
                    res = geolocator.geocode(row[location_col], country_codes=country_code)
                    # if within county
                    if counties_gpd.loc[counties_gpd[admin_col2] == row['County']].contains(Point(res.longitude,res.latitude)).values[0] == True:
                        missing_locations_copy.loc[idx, "Long"] = res.longitude
                        missing_locations_copy.loc[idx, "Lat"] = res.latitude
                        print("\tSuccess for Location name only query")
                        missing_locations_copy.loc[idx, "geocoding_method"] = "Location name only query"
                    else:
                        try:
                            res = geolocator.geocode(row[health_district_col], country_codes=country_code)
                            missing_locations_copy.loc[idx, "Long"] = res.longitude
                            missing_locations_copy.loc[idx, "Lat"] = res.latitude
                            print("\tSuccess for health district")
                            missing_locations_copy.loc[idx, "geocoding_method"] = "Health District name query"
                        except:
                            print('\tFailed')
                            missing_locations_copy.loc[idx, "geocoding_method"] = "Failed"
            except:
                try:
                    res = geolocator.geocode(row[health_district_col], country_codes=country_code)
                    missing_locations_copy.loc[idx, "Long"] = res.longitude
                    missing_locations_copy.loc[idx, "Lat"] = res.latitude
                    print("\tSuccess for health district")
                    missing_locations_copy.loc[idx, "geocoding_method"] = "Health District name"
                except:
                    print('\tFailed')
                    missing_locations_copy.loc[idx, "geocoding_method"] = "Failed"
        i += 1
    print(missing_locations_copy.geocoding_method.value_counts())
    return missing_locations_copy

### import counties

In [25]:
# counties_gpd = gpd.read_file(r"C:\repos\health-equity-access\notebooks\input\lbr_admbnda_adm1_ocha_2019.shp")
counties_gpd = gpd.read_file(os.path.join(input_dir, 'gadm', 'LBR', 'lbr_admbnda_adm1_ocha_20191104.shp'))

In [26]:
counties_gpd[:2]

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
0,2.423034,0.156117,Bomi,LR01,None,None,None,Liberia,LR,2017-12-27,2019-11-04,None,"POLYGON ((-10.86929 6.97176, -10.86563 6.96970..."
1,5.831538,0.712278,Bong,LR02,None,None,None,Liberia,LR,2017-12-27,2019-11-04,None,"POLYGON ((-9.31945 7.43507, -9.31714 7.43368, ..."


In [27]:
new_missing_locations = geocode_missing_locations(lbr_missing, 'Location', 'County', 'health district', counties_gpd, 'ADM1_EN', 'LR')

Geocoding 3, 1 of 72
	Success for health district
Geocoding 88, 2 of 72
	Failed
Geocoding 115, 3 of 72
	Success for health district
Geocoding 157, 4 of 72
	Success for health district
Geocoding 159, 5 of 72
	Success for health district
Geocoding 170, 6 of 72
	Success for Location name and county query
Geocoding 173, 7 of 72
	Success for Another facility with same Location
Geocoding 175, 8 of 72
	Success for Another facility with same Location
Geocoding 176, 9 of 72
	Success for health district
Geocoding 178, 10 of 72
	Success for Another facility with same Location
Geocoding 181, 11 of 72
	Success for Another facility with same Location
Geocoding 182, 12 of 72
	Success for Another facility with same Location
Geocoding 184, 13 of 72
	Success for Location name and county query
Geocoding 188, 14 of 72
	Success for health district
Geocoding 193, 15 of 72
	Success for Another facility with same Location
Geocoding 198, 16 of 72
	Success for Location name and county query
Geocoding 240, 17 of

In [28]:
new_missing_locations.to_csv(os.path.join(out_folder, "health_geocoded_20220609.csv"))